# DataFrameIt - Exemplo 06: Análise Jurídica Avançada

Este notebook demonstra um caso de uso real e avançado do DataFrameIt: análise automatizada de decisões judiciais sobre tratamentos de saúde.

**Conceitos demonstrados:**
- Modelo Pydantic complexo com classes aninhadas
- Campos opcionais e condicionais
- Uso de List e Tuple
- Tipos Literal com múltiplas opções
- Template detalhado e específico de domínio
- Placeholder customizado {sentenca}

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bdcdo/dataframeit/blob/main/example/05_advanced_legal.ipynb)

## 1. Instalação

In [ ]:
!pip install -q dataframeit[google] openpyxl

## 2. Configuração da API Key

In [ ]:
import os

try:
    from google.colab import userdata
    os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
    print("API Key carregada dos Secrets do Colab")
except:
    pass

# os.environ['GOOGLE_API_KEY'] = 'sua-chave-aqui'

if 'GOOGLE_API_KEY' not in os.environ:
    print("AVISO: Configure sua GOOGLE_API_KEY antes de continuar")
else:
    print("API Key configurada com sucesso!")

## 3. Imports

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal
import pandas as pd
from dataframeit import dataframeit

## 4. Definir Modelos Pydantic Complexos

Este exemplo usa modelos aninhados e campos condicionais.

In [ ]:
# Modelo aninhado para tratamento individual
class Tratamento(BaseModel):
    """Informações sobre um tratamento específico pleiteado."""

    nome: str | None = Field(
        ...,
        description="Nome do tratamento pleiteado. 'None' apenas se não foi pleiteado tratamento."
    )

    resultado: Literal['deferido', 'indeferido'] | None = Field(
        None,
        description="Se o tratamento indicado foi deferido ou indeferido. 'None' apenas se não foi pleiteado tratamento."
    )


# Modelo principal com múltiplos campos complexos
class RespostaFinalWrapper(BaseModel):
    """Estrutura completa para análise de decisão judicial sobre saúde."""

    definicao_escopo: Literal["dentro_do_escopo", "escopo_incerto", "fora_do_escopo"] = Field(
        ...,
        description="""Responda 'dentro_do_escopo' se o caso analisado estiver relacionado a casos que cumpram os seguintes requisitos:
1. Trata-se de um pedido de tratamento de saúde.
2. O autor da ação pleiteia tratamento para uma doença rara.

Se o caso não trata de pedido de tratamento de saúde, responda "fora_do_escopo".
Se você estiver em dúvida sobre se o autor da ação tem uma doença rara ou não, responda "escopo_incerto".

Se o caso for classificado como "fora_do_escopo", responda todas as perguntas seguintes como "None"."""
    )

    justificativa: str = Field(
        ...,
        description="Justificativa da avaliação anterior sobre o escopo do caso."
    )

    condicoes_de_saude: list[str] | None = Field(
        None,
        description="Lista de nomes das condições de saúde do paciente (sejam elas raras ou não). Mantenha a mesma redação utilizada pelo magistrado ao se referir a cada doença."
    )

    tratamentos: list[Tratamento] | None = None

    genero_paciente: Literal['masculino', 'feminino', 'outro'] | None = Field(
        None,
        description='Gênero do autor da ação. Responda outro se houver mais de um autor ou não for possível identificar. Responda None se o caso é fora do escopo.'
    )

    justica_gratuita: Literal['nao', 'sim mas foi negado', 'sim e foi aceito'] | None = Field(
        None,
        description='Há menção na decisão ao fato de que o autor da ação requereu justiça gratuita?'
    )

    representacao: Literal['sim', 'nao'] | None = Field(
        None,
        description='O autor da ação é representado por seu responsável/genitor ou seu tutor?'
    )

    danos_morais_pedido: tuple[Literal['sim', 'nao'], float] | None = Field(
        ...,
        description='Se o paciente pediu danos morais e, se sim, quanto. Responda "nao" se não há menção a danos morais na decisão. Responda None se o caso é fora do escopo.'
    )

    danos_morais_concedido: tuple[Literal['sim', 'nao'], float] | None = Field(
        None,
        description='Se o magistrado concedeu o pedido danos morais e, se sim, de que valor. Responda None se não há menção a danos morais na decisão.'
    )

## 5. Definir Template Especializado

Usamos placeholder customizado `{sentenca}` porque é mais semântico para o contexto jurídico.

In [ ]:
TEMPLATE = """
Você é um advogado especialista em direito sanitário.
Irei te apresentar o texto de uma decisão judicial.

Classifique a decisão judicial de acordo com o seguinte formato:

INICIO DAS INSTRUÇÕES DE FORMATAÇÃO
{format}
FIM DAS INSTRUÇÕES DE FORMATAÇÃO

Utilize as informações do texto para preencher os campos.

INICIO DA SENTENÇA
{sentenca}
FIM DA SENTENÇA
"""

## 6. Criar Dados de Exemplo

Criamos dados sintéticos para demonstração.

In [ ]:
dados_sinteticos = {
    'id': [1, 2, 3],
    'texto': [
        """DECISÃO: Trata-se de ação ordinária na qual o autor, diagnosticado com
        Doença de Gaucher (doença rara), pleiteia o fornecimento de tratamento
        específico consistente em terapia de reposição enzimática. O autor é do
        sexo masculino e está representado por sua genitora. Requereu justiça
        gratuita, o que foi deferido. Pleiteia ainda danos morais no valor de
        R$ 50.000,00. DEFIRO o pedido de fornecimento do tratamento, diante da
        comprovação da necessidade médica. Quanto aos danos morais, DEFIRO
        PARCIALMENTE, fixando indenização em R$ 30.000,00.""",

        """SENTENÇA: Cuida-se de pedido de medicamento para tratamento de diabetes.
        O autor não possui doença rara. Trata-se de condição comum. Não houve
        pedido de justiça gratuita. DEFIRO o fornecimento do medicamento pleiteado
        (insulina NPH).""",

        """DECISÃO: A presente ação versa sobre questão administrativa relacionada
        a concurso público. Não há pedido de tratamento de saúde. O autor questiona
        sua nota na prova discursiva. INDEFIRO o pedido por ausência de requisitos."""
    ]
}

df = pd.DataFrame(dados_sinteticos)
print(f"Dataset: {len(df)} decisões judiciais")
df

## 7. Processar com DataFrameIt

In [ ]:
print("Processando decisões judiciais...")

df_final = dataframeit(
    df,
    RespostaFinalWrapper,
    TEMPLATE,
    text_column='texto',
    placeholder='sentenca',      # Placeholder customizado!
    resume=True,
    max_retries=3
)

print("Processamento concluído!")

## 8. Análise dos Resultados

In [ ]:
# Verificar status do processamento
status_counts = df_final['_dataframeit_status'].value_counts()
total = len(df_final)

print(f"Status do processamento:")
print(f"  Processadas: {status_counts.get('processed', 0)}/{total}")
print(f"  Com erro: {status_counts.get('error', 0)}/{total}")

In [ ]:
# Analisar apenas linhas processadas com sucesso
df_sucesso = df_final[df_final['_dataframeit_status'] == 'processed']

if len(df_sucesso) > 0:
    print("Estatísticas das decisões analisadas:")

    # Distribuição por escopo
    print("\nDistribuição por escopo:")
    print(df_sucesso['definicao_escopo'].value_counts())

In [ ]:
# Análise de casos dentro do escopo
casos_escopo = df_sucesso[df_sucesso['definicao_escopo'] == 'dentro_do_escopo']

if len(casos_escopo) > 0:
    print(f"Análise de {len(casos_escopo)} caso(s) dentro do escopo:")

    # Gênero dos pacientes
    print("\nDistribuição por gênero:")
    print(casos_escopo['genero_paciente'].value_counts())

    # Justiça gratuita
    print("\nJustiça gratuita:")
    print(casos_escopo['justica_gratuita'].value_counts())

## 9. Visualizar Resultado Detalhado

In [ ]:
if len(df_sucesso) > 0:
    primeiro = df_sucesso.iloc[0]

    print("Exemplo de resultado detalhado:")
    print(f"\nDefinição de escopo: {primeiro['definicao_escopo']}")
    print(f"Justificativa: {primeiro['justificativa'][:150]}...")

    if primeiro['condicoes_de_saude']:
        print(f"Condições de saúde: {', '.join(primeiro['condicoes_de_saude'])}")

    if primeiro['genero_paciente']:
        print(f"Gênero do paciente: {primeiro['genero_paciente']}")

    if primeiro['justica_gratuita']:
        print(f"Justiça gratuita: {primeiro['justica_gratuita']}")

## 10. Salvar Resultados

In [ ]:
# Descomente para salvar
# df_final.to_excel('decisoes_analisadas.xlsx', index=False)
# print("Resultados salvos em: decisoes_analisadas.xlsx")

## 11. Dicas para Produção

Este exemplo demonstra várias técnicas importantes para produção:

- **Modelo Pydantic complexo** com validação automática
- **Placeholder customizado** específico do domínio (`{sentenca}`)
- **Campos opcionais e condicionais**
- **Uso de List e Tuple** para dados estruturados
- **Template detalhado** com instruções claras
- **Configuração de retry** para resiliência
- **Salvamento automático** de resultados

### Para adaptar este exemplo ao seu caso:
1. Modifique o modelo Pydantic conforme suas necessidades
2. Ajuste o template com instruções específicas do seu domínio
3. Configure `max_retries` baseado na criticidade
4. Implemente pré-processamento de dados se necessário
5. Adicione validações customizadas nos resultados

---

## Próximos Passos

- [06_polars.ipynb](06_polars.ipynb) - Usar com Polars
- [08_rate_limiting.ipynb](08_rate_limiting.ipynb) - Configurar rate limiting